In [ ]:
import pandas as pd

In [ ]:
features = [f'f_{i}' for i in range(300)]
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

for f in features:
    data_types_dict[f] = 'float16'

In [ ]:
features += ['investment_id','time_id']

In [ ]:
# data_types_dict

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict, 
                    index_col = 0
                       )

In [ ]:
train.tail()

In [ ]:
# train = train.sample(1000000)

In [ ]:
example_test = pd.read_csv('/kaggle/input/ubiquant-market-prediction/example_test.csv')

In [ ]:
example_test.tail()

In [ ]:
train.tail()

In [ ]:
train.columns

In [ ]:
# train = train.sample(1000000, random_state=0)

In [ ]:
train.info()

In [ ]:
train.dtypes

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [ ]:
X = train.drop(["target"], axis=1)
y = train["target"]

In [ ]:
del train

In [ ]:
random_state = 13

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=random_state,
                                                    shuffle=False)
del X
del y

In [ ]:
example_test[features]

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {'boosting_type': 'gbdt',
              'objective': 'regression',
              'num_leaves': 40,
              'learning_rate': 0.1,
              'feature_fraction': 0.9
              }
gbm = lgb.train(params,
    lgb_train,
    num_boost_round=200,
    valid_sets=[lgb_train, lgb_eval],
    valid_names=['train','valid'],
    callbacks= [lgb.early_stopping(10)]
   )

In [ ]:
import ubiquant
env = ubiquant.make_env()   
iter_test = env.iter_test()  

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df.drop(['row_id'], axis=1, inplace=True)
    pred = gbm.predict(test_df)
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df) 

In [ ]:
nan